In [2]:
import pandas as pd
import numpy as np
from backtesting import Backtest, Strategy
from data_storage import create_connection
from stockstats import StockDataFrame

In [3]:
connection = create_connection("../database/crypto_billionairs.db")

In [4]:
table = "ETHUSDT_5m_complete_history_long"
table_short = "ETHUSDT_5m_complete_history_short"

In [5]:
df_long = pd.read_sql_query(f'select * from {table}', connection)
df_short = pd.read_sql_query(f'select * from {table_short}', connection)

In [6]:
df_backtesting = pd.DataFrame()
if "5m" not in table:
    df_backtesting["time"] = pd.to_datetime(df_long["open time"], unit='ms')
    df_backtesting[['Open', 'High', 'Low', 'Close', "Volume"]] = df_long[['open', 'high', 'low', 'close', 'volume']]
    df_backtesting = df_backtesting.set_index("time")

else:
    df_backtesting[['Open', 'High', 'Low', 'Close', "Volume"]] = df_long[['open', 'high', 'low', 'close', 'volume']]

In [7]:
df_backtesting["time"] = pd.to_datetime(df_long["open time"], unit='ms')
df_backtesting[['Open', 'High', 'Low', 'Close', "Volume"]] = df_long[['open', 'high', 'low', 'close', 'volume']]
df_backtesting = df_backtesting.set_index("time")

In [8]:
def calculate_osc(data):
    
   
    df_temp = pd.DataFrame()    
    stock = StockDataFrame.retype(data)
    
    df_temp["stochastic_oscillator"] = stock.get('kdjk')
    
    return df_temp["stochastic_oscillator"]

def calucalte_rsi(data):
    
    df_temp = pd.DataFrame()    
    stock = StockDataFrame.retype(data)
    
    df_temp["relative_strength_index"] = stock.get("rsi_30")
    
    return df_temp["relative_strength_index"]

def momentum(df, lag):
    return df["close"].pct_change(periods=lag)

def init_close_long_signal(df):
    
    return df_long['sell_indicator']

def init_buy_signal(df):
    
    return df_long['buy_indicator']

def init_short_signal(df):
    
    return df_short['short_signals']

def init_close_short_signal(df):
    
    return df_short['short_close_signals']
    

In [9]:
class OscRsi_5m_long_short(Strategy):
   
    lag = 4
    
    def init(self):
        # compute the rsi and stochastic oscillator with stockstats and return the buy signal of the current row
        
        self.osc = self.I(calculate_osc, self.data.df)
        self.rsi = self.I(calucalte_rsi, self.data.df)
        self.buy_init = self.I(init_buy_signal, self.data.df)
        self.close_buy_init = self.I(init_close_long_signal, self.data.df)
        self.short_init = self.I(init_short_signal, self.data.df)
        self.close_short_init = self.I(init_close_short_signal, self.data.df)
        self.data.df[['Open', 'High', 'Low', 'Close', "Volume",]] = self.data.df[['open', 'high', 'low', 'close', "volume"]]
        self.returns = self.I(momentum, self.data.df, self.lag)
        self.data.df.drop(self.data.df.columns.difference(['Open', 'High', 'Low', 'Close', "Volume", "kdjk", "rsi_30"]), 1, inplace=True)
        
       
    
    def next(self):
        
        if self.buy_init[-1] == 1 and self.position.is_long is False:
            self.position.close()
            self.buy()
            
        # elif self.close_buy_init[-1] == 1 and self.position.is_long:
        #      self.position.close()
             
             
        elif self.short_init == 1 and self.position.is_short is False:
            self.position.close()
            self.sell()


In [10]:
bt = Backtest(df_backtesting, OscRsi_5m_long_short, cash=10_000, commission=.001)
stats = bt.run()
stats

Start                     2021-03-01 00:00:00
End                       2022-03-31 23:55:00
Duration                    395 days 23:55:00
Exposure Time [%]                     96.8533
Equity Final [$]                       135362
Equity Peak [$]                        143316
Return [%]                            1253.62
Buy & Hold Return [%]                 131.947
Return (Ann.) [%]                     1003.87
Volatility (Ann.) [%]                 1724.56
Sharpe Ratio                         0.582102
Sortino Ratio                          18.184
Calmar Ratio                          27.6643
Max. Drawdown [%]                    -36.2877
Avg. Drawdown [%]                    -1.97945
Max. Drawdown Duration       74 days 04:10:00
Avg. Drawdown Duration        0 days 21:17:00
# Trades                                   15
Win Rate [%]                              100
Best Trade [%]                        106.906
Worst Trade [%]                      0.224486
Avg. Trade [%]                    

In [45]:
bt.plot()

Row(id='3663', ...)

In [46]:
# %%time 
# stats = bt.optimize(lag=[12, 24, 48, 96, 120, 144, 168],
#                     sell_ = np.arange(60, 90, 3).tolist(),#sell_threshold = np.arange(0, -0.15, -0.01).tolist(),
#                     rsi_lower_bound = np.arange(10, 40, 3).tolist(),
#                     rsi_upper_bound = np.arange(12, 50, 3).tolist(),
#                     #osc_lower_bound = np.arange(10, 90, 5).tolist(),
#                     #osc_upper_bound = np.arange(10, 100, 5).tolist(),
#                     #maximize='Equity Final [$]')#,
#                     constraint= lambda param: param.rsi_lower_bound < param.rsi_upper_bound)
# stats

In [11]:
stats._strategy

<Strategy OscRsi_5m_long_short>

In [12]:
stats['_trades']

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,5,3583,20299,1875.81394,3881.18,10026.83030,1.069064,2021-03-13 12:05:00,2021-05-10 20:15:00,58 days 08:10:00
1,-5,20299,22803,3877.29882,2006.68,9353.09410,0.482454,2021-05-10 20:15:00,2021-05-19 12:55:00,8 days 16:40:00
2,14,22803,22813,2008.68668,2560.92,7731.26648,0.274923,2021-05-19 12:55:00,2021-05-19 13:45:00,0 days 00:50:00
3,-14,22813,32122,2558.35908,2261.26,4159.38712,0.116129,2021-05-19 13:45:00,2021-06-20 21:30:00,32 days 07:45:00
4,18,32122,54743,2263.52126,3491.40,22101.81732,0.542464,2021-06-20 21:30:00,2021-09-07 15:05:00,78 days 17:35:00
5,-18,54743,54744,3487.90860,3035.87,8136.69480,0.129602,2021-09-07 15:05:00,2021-09-07 15:10:00,0 days 00:05:00
6,23,54744,54752,3038.90587,3454.00,9547.16499,0.136593,2021-09-07 15:10:00,2021-09-07 15:50:00,0 days 00:40:00
7,-23,54752,56117,3450.54600,3442.80,178.15800,0.002245,2021-09-07 15:50:00,2021-09-12 09:35:00,4 days 17:45:00
8,23,56117,77683,3446.24280,4128.07,15682.02560,0.197847,2021-09-12 09:35:00,2021-11-26 08:45:00,74 days 23:10:00
9,-23,77683,77684,4123.94193,4056.90,1541.96439,0.016257,2021-11-26 08:45:00,2021-11-26 08:50:00,0 days 00:05:00


In [49]:
trades = pd.DataFrame(stats['_trades'])

In [51]:
trades.to_excel(f"trades_{table}_short.xlsx")